# Neural Network
### Mathematical Notation:

$$z = \mathbf{w} \mathbf{x} + b$$
$$a = f(z)$$


In [ ]:
import requests, os, gzip
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
mnist_address = [
    "https://github.com/entropicemergence/tiny_llm_server/releases/download/v0.1.0/t10k-images-idx3-ubyte.gz",
    "https://github.com/entropicemergence/tiny_llm_server/releases/download/v0.1.0/t10k-labels-idx1-ubyte.gz",
    "https://github.com/entropicemergence/tiny_llm_server/releases/download/v0.1.0/train-images-idx3-ubyte.gz",
    "https://github.com/entropicemergence/tiny_llm_server/releases/download/v0.1.0/train-labels-idx1-ubyte.gz"
]


os.makedirs("mnist_data", exist_ok=True)
mnist_folder = "mnist_data"

def download_mnist(address):
    for address in mnist_address:
        response = requests.get(address)
        file_name = address.split("/")[-1]
        path = os.path.join(mnist_folder, file_name)
        with open(path, "wb") as f:
            f.write(response.content)
download_mnist(mnist_address)

In [ ]:
with gzip.open(os.path.join(mnist_folder, "train-labels-idx1-ubyte.gz"), "rb") as f:
    train_labels = np.frombuffer(f.read(), np.uint8, offset=8)
with gzip.open(os.path.join(mnist_folder, "train-images-idx3-ubyte.gz"), "rb") as f:
    train_images = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1, 28, 28)
with gzip.open(os.path.join(mnist_folder, "t10k-labels-idx1-ubyte.gz"), "rb") as f:
    test_labels = np.frombuffer(f.read(), np.uint8, offset=8)
with gzip.open(os.path.join(mnist_folder, "t10k-images-idx3-ubyte.gz"), "rb") as f:
    test_images = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1, 28, 28)

In [ ]:
fig, ax = plt.subplots(2, 5, figsize=(15, 6))
for i in range(10):
    ax[i//5, i%5].imshow(train_images[i], cmap="gray")
    ax[i//5, i%5].set_title(f"Label: {train_labels[i]}")
    ax[i//5, i%5].axis("off")
plt.tight_layout()
plt.show()



In [ ]:
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

# print(train_images[1])
print (test_labels[0:20])

In [ ]:

scallar = np.array([1])

vector = np.array([1, 2, 3])

matrix = np.array([[1, 2, 3], [4, 5, 6]])

print(scallar.shape)
print(vector.shape)
print(matrix.shape)

In [ ]:
input_size = 10
output_size = 4
w = np.random.randn(output_size, input_size) * np.sqrt(2 / input_size)
print(w)

In [ ]:

from torch._refs import exponential


class LinearLayer():
    def __init__(self, input_size, output_size):
        self.W = np.random.randn(output_size, input_size) * np.sqrt(2 / input_size)
        self.B = np.zeros(output_size)
    def relu_activation(self, input):
        return np.maximum(0, input)
    def forward(self, input):
        output = np.dot(self.W, input) + self.B
        output = self.relu_activation(output)
        return output

class NumpyFullyConnectedNetwork():
    def __init__(self):
        self.layers = []
        self.layers.append(LinearLayer(28 * 28, 196))
        self.layers.append(LinearLayer(196, 32))
        self.layers.append(LinearLayer(32, 10))
    def softmax(self, input):
        exponential = input - np.max(input)
        exponential = np.exp(exponential)
        return exponential / np.sum(exponential)
    def forward(self, input):
        for layer in self.layers:
            input = layer.forward(input)
        return self.softmax(input)
    

print (train_images[0].shape)
train_images_flat = train_images.reshape(-1, 28*28)
# print (train_images_flat[0])
train_images_flat_scale = (train_images_flat / 255.0) - 0.5 
# print (train_images_flat_scale[0].shape)



numpy_net = NumpyFullyConnectedNetwork()
output = numpy_net.forward(train_images_flat_scale[0])
print (output)
print (np.argmax(output))



fig, ax = plt.subplots(2, 5, figsize=(15, 6))
for i in range(10):
    output = numpy_net.forward(train_images_flat_scale[i])
    prediction = np.argmax(output)
    ax[i//5, i%5].imshow(train_images[i], cmap="gray")
    ax[i//5, i%5].set_title(f"Label: {train_labels[i]}, Prediction: {prediction}")
    ax[i//5, i%5].axis("off")
plt.tight_layout()
plt.show()





In [ ]:
print (28*28/4)

# Fully connected network

https://adamharley.com/nn_vis/mlp/3d.html

# Membuat Neural Network menggunakan pytorch

In [ ]:
## Batch training

# Convolutional Neural Network

https://adamharley.com/nn_vis/cnn/3d.html